<a href="https://colab.research.google.com/github/ProfAI/tutorials/blob/master/Reti%20Neurali%20per%20la%20Sentiment%20Analysis%20con%20Keras/imdb_sentiment_analysis_with_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Come creare una Rete Neurale per la Sentiment Analysis con Keras